<a href="https://colab.research.google.com/github/shubham-20120/knn-project/blob/main/movies_recommendation_in_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

If you are willing to clone this project, please download dataset from [here](https://www.kaggle.com/rounakbanik/the-movies-dataset/download), then import "rating.csv" and "movies_metadata.csv"

## Importing important libraries

In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors
!pip install fuzzywuzzy
from fuzzywuzzy import process
import time

/usr/local/lib/python3.7/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


## Import Dataset

In [4]:
rating = pd.read_csv('/content/drive/MyDrive/movie dataset/ratings.csv', usecols=['userId', 'movieId', 'rating'], dtype={'movieId':'int32', 'rating':'float32', 'userId':'int32'})
movies = pd.read_csv('/content/drive/MyDrive/movie dataset/movies_metadata.csv', usecols=['id', 'title'], dtype={'movieId':'int32','title':'str'})

## Visualizing Data

In [5]:
movies.head()

,id,title
0,862,Toy Story
1,8844,Jumanji
2,15602,Grumpier Old Men
3,31357,Waiting to Exhale
4,11862,Father of the Bride Part II


In [6]:
rating.head()

,userId,movieId,rating
0,1,110,1.0
1,1,147,4.5
2,1,858,5.0
3,1,1221,5.0
4,1,1246,5.0


In [7]:
rating.shape

(26024289, 3)

## Dividing Dataset in small chunks
###### As dataset is too big, this will consume too much RAM, and if we are using [Google Colab](https://colab.research.google.com/), this big data will terminate our session

In [ ]:
chunk_size = 500000
chunks = [x for x in range(0, rating.shape[0], chunk_size)]
len(chunks)

In [ ]:
div = rating[0:int(chunks[1])]
dataMat = div.pivot(index='userId', columns='movieId',values='rating')
temp = int(chunks[1])
print('hold... this will take couple of minutes .... \n')
for i in range(2, len(chunks)-1):
    print(int(temp/500000), '/', len(chunks))
    div = rating[temp:int(chunks[i])]
    temp=int(chunks[i])
    dataMat.append(div.pivot(index='userId', columns='movieId',values='rating'))


# At the end of this cell, we will get a 'data matrix - dataMat'
# which consists of userId as row index and movieId as column index
# and this matrix will hold movies ratings, to visialize matrix un-comment below command
#dataMat

##### As running above cell will take too much time, you can export a csv consisting of 'dataMat' for future reference

In [ ]:
np.savetxt("dataMat.csv", dataMat, delimiter=",")

### Importing previously saved data-Matrix (if not saved, proceed with dataMat)

In [8]:
data = pd.read_csv('/content/drive/MyDrive/movie dataset/dataMat.csv')
data=csr_matrix(data.values)

## Defining out KNN model

In [9]:
model = NearestNeighbors(metric='cosine', n_neighbors=10, algorithm='brute')

#### Training model

In [10]:
model.fit(data)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                 radius=1.0)

# Driving function

In [11]:
def engine(searchQuery, n):
    print('\nsearching for reccomendations ....\n')
    try:
        movie_idx = process.extractOne(searchQuery,movies['title'])[2]
    except:
        print('\nmovie not found :(\n\n')
    try:
        distances, indices=model.kneighbors(data[movie_idx], n_neighbors=n)
        print('movies found...\n')
        for i in indices:
            print(movies['title'][i].where(i!=movie_idx))
                
    except:
        print('\nno similar movies found :(\n\n')

#### user queries

In [12]:
print('\n\n!!! Welcome to Hollywood World !!!\n\n')
movieName = input('Enter movie name: ')
n = int(input('\nEnter No. of Movies you want: '))
engine(movieName, n)



!!! Welcome to Hollywood World !!!


Enter movie name: jimanji

Enter No. of Movies you want: 10

searching for reccomendations ....

movies found...

1215                                   NaN
692                            Sunset Park
4915                              Galaxina
2812                      The Razor's Edge
2898                         Bustin' Loose
1356                                Scream
65      Lawnmower Man 2: Beyond Cyberspace
5119                        Tender Cousins
3472                          Center Stage
4546                       Three Fugitives
Name: title, dtype: object
